In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

# Simulate 

In [ ]:
from helpers import compute

In [ ]:
df = compute(80)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(df['Start'], range(len(df)), label="Arrived")
plt.step(df['T22'], range(len(df)),label="Served")
plt.title("Number of requests arrived and served")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.step(df['Start'][100:150], range(50), label="Arrived")
plt.step(df['T22'][100:150], range(50), label="Served")
plt.title("Number of requests arrived and served (zoomed)")
plt.legend()
plt.show()

### average response time (event average)

In [ ]:
#time difference between end of task, and beginning of task
# for each type of task
df['T1-Serv'] = df['T12']-df['Start'] # serv time for type 1
df['T2-Serv'] = df['T22']-df['T12']  # serv time for type2

In [ ]:
plt.bar([1],[df['T1-Serv'].mean()], label="Type 1")
plt.bar([2],[df['T2-Serv'].mean()], label="Type 2")
plt.title("Average response time for a job")
plt.xticks([])
plt.ylabel('Miliseconds')
plt.legend();

### Average number of jobs served per second

In [ ]:
served = []
for sample_start_time in (df.values.max()-1000)*np.random.sample(100):
    served.append(df[(df['T21'] > sample_start_time) & (df['T22'] < sample_start_time+1000)].count()['Start'])
print("Average number of Type 1 jobs served per second: {}".format(np.mean(served)))

In [ ]:
served = []
for sample_start_time in (df.values.max()-1000)*np.random.sample(50):
    served.append(df[(df['T11'] > sample_start_time) & (df['T12'] < sample_start_time+1000)].count()['Start'])
print("Average number of Type 2 jobs served per second: {}".format(np.mean(served)))

As we see, the number of jobs served per second is in both cases close to 80, as the server can keep up with the requests rate, of 80 requests/seconds. So the queue being most of the time almose empty, the rate is obviously close to 80.

# 2

In [ ]:
lambdas = np.arange(25, 250, 25)
queues = []

In [ ]:
items = []
#for l in lambdas: 
#    it = compute(l)
#    items.append(it)

In [ ]:
#for i in range(len(items)): 
#    df = pd.DataFrame(items[i], columns = ['Start', 'T11', 'T12', 'T21', 'T22'])
#    df.to_csv("lambda_"+str(25*(i+1)), index=False)

# 3 Remove Transients

## With transient present
### $\lambda = 60$

In [ ]:
from helpers import get_type_i_in_queue, plot_type_i_in_queue

In [ ]:
df60 = compute(60)
interesting_times60, type1_in_queue_60, type2_in_queue_60 = get_type_i_in_queue(df60)

In [ ]:
plot_type_i_in_queue(interesting_times60, type1_in_queue_60, type2_in_queue_60)

#### CI for median

In [ ]:
from helpers import ci_median

In [ ]:
l,h = ci_median(len(interesting_times60))
a = np.sort(type1_in_queue_60)
b = np.sort(type2_in_queue_60)
print("The CI for median with lambda=60 are, at 95% confidence, between\n\ttype1: {} and {}\n\ttype2: {} and {}".format(a[l], a[h], b[l], b[h]))

#### CI for mean

In [ ]:
from helpers import ci_mean_large_n

In [ ]:
ci_means_type1 = ci_mean_large_n(type1_in_queue_60)
ci_means_type2 = ci_mean_large_n(type2_in_queue_60)
print("The CI for median with lambda=60 are, at 95% confidence,\
between\n\ttype1: {:.3f} and {:.3f}\n\
\ttype2: {:.3f} and {:.3f}".format(ci_means_type1[0], ci_means_type1[1], 
                                   ci_means_type2[0], ci_means_type2[1]))

## $\lambda = 160$

#### CI for median

In [ ]:
df160 = compute(160)
interesting_times160, type1_in_queue_160, type2_in_queue_160 = get_type_i_in_queue(df160)
plot_type_i_in_queue(interesting_times160, type1_in_queue_160, type2_in_queue_160)

In [ ]:
l,h = ci_median(len(interesting_times60))
type1_160_sorted = np.sort(type1_in_queue_160)
type2_160_sorted = np.sort(type2_in_queue_160)
print("The CI for median with lambda=60 are, at 95% confidence,\
between\n\ttype1: {} and {}\
\n\ttype2: {} and {}".format(type1_160_sorted[l], type1_160_sorted[h], 
                             type2_160_sorted[l], type2_160_sorted[h]))

#### CI for mean

In [ ]:
ci_means_type1 = ci_mean_large_n(type1_in_queue_160)
ci_means_type2 = ci_mean_large_n(type2_in_queue_160)
print("The CI for median with lambda=160 are, at 95% confidence,\
between\n\ttype1: {:.3f} and {:.3f}\n\
\ttype2: {:.3f} and {:.3f}".format(ci_means_type1[0], ci_means_type1[1], 
                                   ci_means_type2[0], ci_means_type2[1]))

## With the transient removed
For $\lambda = 60$, the analysis is the same as before, as there seems to be no transient. Indeed, the buffer is most of the time empty or nearly empty, steadily accross time. Thus we do not repeat the calculation.

For $\lambda = 160$, we use as transient period the time until the buffer size reaches the mean of both objects (that is 97.884). Everything before this threshold is reached is considered transient period

In [ ]:
# Computing moment the threshold is first crossed
buffer_size = np.array(type1_in_queue_160 + type2_in_queue_160)
threshold = np.mean(buffer_size)
threshold_crossing = np.argwhere(buffer_size > threshold)[0,0]

#### CI for median

In [ ]:
#computing median on reduced array
med_l_ind, med_h_ind = ci_median(len(interesting_times160[threshold_crossing:]))
#lower and higher for type 1
med_l_type1 = type1_160_sorted[threshold_crossing+med_l_ind]
med_h_type1 = type1_160_sorted[threshold_crossing+med_h_ind]

med_l_type2 = type2_160_sorted[threshold_crossing+med_l_ind]
med_h_type2 = type2_160_sorted[threshold_crossing+med_h_ind]
print("CI for median, lambda=160, no transient, at 95% level:\n\
\tType1: {} - {}\n\
\tType2: {} - {}".format(med_l_type1, med_h_type1, 
                         med_l_type2, med_h_type2))

#### CI for mean

In [ ]:
ci_mean_l_type1, ci_mean_h_type1 = ci_mean_large_n(type1_in_queue_160[threshold_crossing:])
ci_mean_l_type2, ci_mean_h_type2 = ci_mean_large_n(type2_in_queue_160[threshold_crossing:])

In [ ]:
print("CI for mean, lambda=160, no transient, at 95% level:\n\
\tType1: {:.3f} - {:.3f}\n\
\tType2: {:.3f} - {:.3f}".format(ci_mean_l_type1, ci_mean_h_type1, 
                         ci_mean_l_type2, ci_mean_h_type2))